In [100]:
class A(object):
    def __init__(self, i):
        self.i = i
    def run(self, value):
        return self.i * value

class B(A):
    def __init__(self, i, j):
        super(B, self).__init__(i)
        self.j = j
    def run(self, value):
        return super(B, self).run(value) + self.j

In [101]:
class Logger(object):
    def __init__(self, name, *args, **kwargs):
        self.name = name
    
    def run_logged(self, value):
        print "Running", self.name, "with info", self.info()
        return self.run(value)

class BLogged(B, Logger):
    def __init__(self, i, j):
        B.__init__(self, i, j) # Bad...
        Logger.__init__(self, "B") # Even worse...
    def info(self):
        return 42

In [102]:
b = B(3, 4)

In [103]:
B.__mro__

(__main__.B, __main__.A, object)

In [104]:
BLogged.__mro__

(__main__.BLogged, __main__.B, __main__.A, __main__.Logger, object)

In [105]:
bl = BLogged(3, 4)

In [106]:
bl.run(3)

13

In [107]:
dir(bl)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'i',
 'info',
 'j',
 'name',
 'run',
 'run_logged']

In [108]:
Logger.__mro__

(__main__.Logger, object)

In [109]:
s = super(BLogged, bl)

In [110]:
dir(s)

['__class__',
 '__delattr__',
 '__doc__',
 '__format__',
 '__get__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__self_class__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__thisclass__',
 'i',
 'j',
 'name']

In [111]:
type(s)

super

It looks fine. So what is the problem of hard coding `B.__init__` and `Logger.__init__`?

Let's see some example.

Suppose you want another class inheritate from `B` and other `Base` class, let's see what will happend if you try to inheritate both `B` and `Base`:

In [112]:
class Base(object):
    
    def __init__(self, *args, **kwargs):
        self.base = "this is from Base!"

class Child(Base, BLogged): pass


In [113]:
child = Child(1, 2)

In [114]:
dir(child)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'base',
 'info',
 'run',
 'run_logged']

In [115]:
print(child.base)

this is from Base!


In [116]:
# twist it a bit:
class Child(BLogged, Base):pass

In [117]:
child = Child(1, 2)
print(child.base)

AttributeError: 'Child' object has no attribute 'base'

What!? the `Base` is gone!

This is because we hard code `B.__init__` and `Logged.__init__`.

The initiation of `Child` instance (the function call to `Child.__init__`) is as following:

1. Since `Child` does not implement `__init__`, `BLogged.__init__` is called since it is the next base class in the `mro`
2. In `BLogged.__init__`, `B.__init__` and `Logger.__init__` are called. The the initiation is done.
3. `Base.__init__` never get the chace to be called. That is why the `base` attribute is gone.

## The Fix

Here is a fixed version of above classes. (with `super`!)

In [138]:
class AFixed(object):
    def __init__(self, i, *args, **kwargs):
        self.i = i
        super(AFixed, self).__init__(*args, **kwargs)
    
    def run(self, value):
        return self.i * value

class BFixed(AFixed):
    def __init__(self, i, j, *args, **kwargs):
        self.j = j
        super(BFixed, self).__init__(i, *args, **kwargs)
    
    def run(self, value):
        return super(B, self).run(value) + self.j

class LoggerFixed(object):
    def __init__(self, name, *args, **kwargs):
        self.name = name
        super(LoggerFixed, self).__init__(*args, **kwargs)
    
    def run_logged(self, value):
        print "Running", self.name, "with info", self.info()
        return self.run(value)

class BLoggedFixed(BFixed, LoggerFixed):
    def __init__(self, i, j, *args, **kwargs):
        super(BLoggedFixed, self).__init__(i, j, *args, **kwargs)
    def info(self):
        return 42

class ChildV1(BLoggedFixed, Base): pass
class ChildV2(Base, BLoggedFixed): pass

In [139]:
child1 = ChildV1(1, 2, "name")

In [140]:
ChildV1.__mro__

(__main__.ChildV1,
 __main__.BLoggedFixed,
 __main__.BFixed,
 __main__.AFixed,
 __main__.LoggerFixed,
 __main__.Base,
 object)

In [141]:
child1.base

'this is from Base!'

In [142]:
child2 = ChildV2(1, 2, "name")
child2.base

'this is from Base!'

## Lesson Learnd

1. use `super` to maintain the `__mro__`
2. Mutiple inherientance is a bad idea. The mro tree can be hard to trace.

In [48]:
class A(object):
    def __init__(self, i, **kwargs):
        """
        A
        """
        super(A, self).__init__(**kwargs)
        self.i = i
    def run(self, value):
        return self.i * value

class B(A):
    def __init__(self, j, **kwargs):
        """
        B
        """
        super(B, self).__init__(**kwargs)
        self.j = j
    def run(self, value):
        return super(B, self).run(value) + self.j

class Logger(object):
    def __init__(self, name, **kwargs):
        """
        C
        """
        super(Logger,self).__init__(**kwargs)
        self.name = name
    def run_logged(self, value):
        print "Running", self.name, "with info", self.info()
        return self.run(value)

class BLogged(B, Logger):
    def __init__(self, **kwargs):
        """
        D
        """
        super(BLogged, self).__init__(name="B", **kwargs)
    def info(self):
        return 42

b = BLogged(i=3, j=4)

In [49]:
b.name

'B'

In [50]:
BLogged.__mro__

(__main__.BLogged, __main__.B, __main__.A, __main__.Logger, object)

In [51]:
s = super(BLogged, b)

In [52]:
s.__init__.__doc__

'\n        B\n        '